## Model (One-Shot)

The model consists of the following elements:

* agents $a_1, a_2, \ldots, a_n \in A$ 

* projects $p_1, p_2, \ldots, p_m \in P$

* clusters $c_1, c_2, \ldots, c_s \in C$

* agents budgets $b_1, b_2, \ldots, b_n \in \mathbb{R}^+$

* agents private beliefs $l_1, l_2, \ldots, l_n \in \mathbb{R}^{m+}$

* agents have public voices $v_1, v_2, \ldots, v_n \in \mathbb{R}$ such that $v_i \le b_i, \ \forall a \in A$ 

* each pair of agents $a_i, a_j$ have a set of affiliations over clusters $f_{ij} \in \mathbb{R}^{s+}$

* agent-agent-cluster affiliation matrix $F$

* each cluster $c_i$ has a set of affiliations over project $p_j$, $g_{ij} \in \mathbb{R}^{m+}$

* cluster-project affiliation matrix $G$


In order to accomodate the equations in the SBT paper, we need a function to reduce the set of agent-agent affiliations over clusters to a single value. For now, to get a single agent-agent affiliation, we simply take the sum of the values in the corresponding agent-agent affiliation vector.

In order to account for affiliations of clusters to projects, we can transform the agent-agent-cluster matrix to a agent-agent-project matrix, and then reduce those values (as before, for now, we simply take the sum of the values).

This process gives a way to model the agents' public voices from their private beliefs, and Wgives us a setting from which we can plug values into the equations in the SBT paper




### Notes

A potential goal is to have agents' private beliefs equal their public voices (incentive-compatiblity).

It is possible to weight the importance every agent gives to every cluster in order to get a more refined consolidation of agent-agent affiliation (the same is true of clusters giving importance to projects).

We are multiplying agents' affiliation weights by the other agents' private values, not public values.

Prior research on homophily, diffusion, and learning in networks can be leveraged in this environment.

We can use matrix factorization and other decomposition methods to make some interesting insights (given real-world data). 

In [1]:
import numpy as np    

# !pip install import_ipynb
import import_ipynb

In [2]:
%%capture  
from SBT import *

In [3]:
num_agents = 100
num_projects = 10
num_clusters = 20

In [4]:
agents = list(map(str, range(num_agents)))

In [5]:
def create_clusters_to_agents_dict():
    """
    :return: a dictionary mapping clusters to the agents in those clusters
    """
    # initialize a dictionary mapping each cluster id to the agents in that cluster
    clusters_to_agents_dict = {}
    # array describing the fraction of agents in each cluster 
    fraction_of_agents_in_each_cluster = [0.20] * num_clusters
    # populate the dictionary
    for cluster in range(num_clusters):
        # get the fraction of agents in this cluster
        fraction_of_agents_in_this_cluster = fraction_of_agents_in_each_cluster[cluster]
        # randomly select agents to include in this cluster
        agents_in_cluster = np.random.choice(agents, size=int(fraction_of_agents_in_this_cluster * num_agents), replace=False)
        # assign agents to the cluster
        clusters_to_agents_dict[cluster] = agents_in_cluster
        
    return clusters_to_agents_dict

## Basic Model Components 

In [6]:
# array describing the maximum amount that each agent can donate 
agent_budgets = np.zeros([num_agents])
# matrix describing the affiliation of each agent with each other agent on each cluster
# so an entry (i, j, k) is an array of length num_clusters representing the affiliation of agents i and j on each cluster
agent_agent_cluster_affiliation_matrix = np.zeros([num_agents, num_agents, num_clusters])
# matrix describing the affiliation of each cluster to each project
cluster_project_affiliation_matrix = np.zeros([num_projects, num_clusters])
# a matrix describing each agent's private belief (how much they think that they should donate to each project)
agent_private_beliefs = np.zeros([num_agents, num_projects])
# a matrix describing each agent's public belief (how much they actually donate to each project)
agent_public_beliefs = np.zeros([num_agents, num_projects])

Strategy to create agent private beliefs: start out with some (maybe random) vector to denote an agent's private beliefs, and then account for agent affiliations on top of that.

In [7]:
def create_agent_budgets():
    """
    :return: an array of the agents' budgets
    """

    # create agent budgets
    agent_budget_lower_bound = 0
    agent_budget_upper_bound = 100
    agent_budgets = np.random.uniform(low=agent_budget_lower_bound, high=agent_budget_upper_bound, size=[num_agents])
    
    return agent_budgets

In [8]:
def create_agent_private_beliefs():
    """
    :return: a matrix of size num_agents x num_projects, where entry (i, j) corresponds to agent i's private belief about project j
    """

    # alternative 1
    """
    # create agent private beliefs
    agent_private_belief_lower_bound = 0
    agent_private_belief_upper_bound = 10
    agent_private_beliefs = np.random.uniform(low=agent_private_belief_lower_bound, high=agent_private_belief_upper_bound, size=[num_agents, num_projects])
    
    return agent_private_beliefs
    """
    
    # alternative 2
    # create agent private beliefs
    agent_private_belief_lower_bound = 0.1
    agent_private_belief_upper_bound = 100
    agent_private_beliefs = np.random.uniform(low=agent_private_belief_lower_bound, high=agent_private_belief_upper_bound, size=[num_agents, num_projects])
    
    # set each private belief to 0 with probability e^-(private belief/temperature_scaling_factor), 
    # so that the higher the value, the lower the probability of being set to zero
    temperature_scaling_factor = 1
    exponentiated_agent_private_beliefs = np.exp(-agent_private_beliefs / temperature_scaling_factor)
    mask_agent_private_beliefs = np.random.binomial(n=np.full_like(exponentiated_agent_private_beliefs, fill_value=1, dtype=int), 
                                                                p=exponentiated_agent_private_beliefs,
                                                                size=exponentiated_agent_private_beliefs.shape)

    # apply the 0-1 mask to the affiliation matrix
    agent_private_beliefs *= mask_agent_private_beliefs
    
    # make sure each agent has a positive sum over beliefs
    for num_agent in range(num_agents):
        if sum(agent_private_beliefs[num_agent]) == 0:
            index_to_modify = np.random.choice(range(num_projects))
            agent_private_beliefs[num_agent, index_to_modify] = np.random.uniform(low=agent_private_belief_lower_bound, high=agent_private_belief_upper_bound)

    return agent_private_beliefs

In [9]:
def normalize_agent_beliefs_to_equal_budget(agent_private_beliefs, agent_budgets):
    """
    :param agent_private_beliefs: an array of the agents' private beliefs
    :param agent_budgets: an array of the agents' budgets
    :return: an array of the agents' private beliefs normalized so that the sum of each agent's beliefs equals their budget
    """

    # assuming agents use up all their budgets, normalize their private beliefs to equal their budgets
    agent_sum_beliefs = agent_private_beliefs.sum(axis=1)
    agent_sum_beliefs_normalizing_factor = (agent_budgets / agent_private_beliefs.sum(axis=1))[:, None]
    agent_private_beliefs = agent_private_beliefs * agent_sum_beliefs_normalizing_factor
    
    return agent_private_beliefs

In [10]:
def create_agent_agent_cluster_affiliation_matrix():
    """
    :return: a matrix of size num_agents x num_agents x num_clusters, where entry (i, j, k) corresponds to 
    agent i's and agent's j's affiliation on cluster k
    """

    
    # alternative 1
    """
    # create the agent-agent-cluster affiliation matrix
    agent_agent_cluster_affiliation_lower_bound = 0
    agent_agent_cluster_affiliation_upper_bound = 10
    agent_agent_cluster_affiliation_matrix = np.random.uniform(low=agent_agent_cluster_affiliation_lower_bound, high=agent_agent_cluster_affiliation_upper_bound, size=[num_agents, num_agents, num_clusters])

    # make matrix symmetric
    for row in range(num_agents):
        for column in range(row, num_agents):
            agent_agent_cluster_affiliation_matrix[column, row] = agent_agent_cluster_affiliation_matrix[row, column]
            
            
    return agent_agent_cluster_affiliation_matrix
    """
    
    # alternative 2
    # for each cluster, let each agent-agent affiliation be drawn from an exponential function with its own cluster-specific parameter
    agent_agent_cluster_parameter_affiliation_lower_bound = 0.1
    agent_agent_cluster_parameter_affiliation_upper_bound = 10
    # agent_agent_cluster_exponential_parameters = np.random.uniform(low=agent_agent_cluster_parameter_affiliation_lower_bound, 
    #                                                                 high=agent_agent_cluster_parameter_affiliation_upper_bound, 
    #                                                                 size=num_clusters)
    agent_agent_cluster_parameter_affiliation_parameter = np.mean(agent_agent_cluster_parameter_affiliation_lower_bound + agent_agent_cluster_parameter_affiliation_upper_bound)
    agent_agent_cluster_exponential_parameters = np.random.exponential(scale=agent_agent_cluster_parameter_affiliation_parameter, 
                                                                       size=num_clusters)
    
    # create the agent-agent-cluster affiliation matrix
    agent_agent_cluster_affiliation_matrix = np.zeros(shape=[num_agents, num_agents, num_clusters])
    for num_cluster in range(num_clusters):
        current_cluster_parameter = agent_agent_cluster_exponential_parameters[num_cluster]
        agent_agent_cluster_affiliation_matrix[:, :, num_cluster] = np.random.exponential(scale=current_cluster_parameter, size=[num_agents, num_agents])

    # set each value in the agent-agent-cluster affiliation matrix to 0 with probability e^-(value/temperature_scaling_factor), 
    # so that the higher the value, the lower the probability of being set to zero
    temperature_scaling_factor = np.mean(agent_agent_cluster_parameter_affiliation_lower_bound + agent_agent_cluster_parameter_affiliation_upper_bound)
    exponentiated_agent_agent_cluster_affiliation_matrix = np.exp(-agent_agent_cluster_affiliation_matrix / temperature_scaling_factor)
    mask_agent_agent_cluster_affiliation_matrix = np.random.binomial(n=np.full_like(exponentiated_agent_agent_cluster_affiliation_matrix, fill_value=1, dtype=int), 
                                                                p=exponentiated_agent_agent_cluster_affiliation_matrix,
                                                                size=exponentiated_agent_agent_cluster_affiliation_matrix.shape)

    # apply the 0-1 mask to the affiliation matrix
    agent_agent_cluster_affiliation_matrix *= mask_agent_agent_cluster_affiliation_matrix

    # make matrix symmetric
    for row in range(num_agents):
        for column in range(row, num_agents):
            agent_agent_cluster_affiliation_matrix[column, row] = agent_agent_cluster_affiliation_matrix[row, column]

    return agent_agent_cluster_affiliation_matrix

In [11]:
def create_cluster_project_affiliation_matrix():
    """
    :return: a matrix of size num_clusters x num_projects, where entry (i, j) corresponds to clusters i's and project j's affiliation
    """

    
    # alternative 1
    """
    # create the cluster-project affiliation matrix
    cluster_project_affiliation_lower_bound = 0
    cluster_project_affiliation_upper_bound = 10
    cluster_project_affiliation_matrix = np.random.uniform(low=cluster_project_affiliation_lower_bound, high=cluster_project_affiliation_upper_bound, size=[num_projects, num_clusters])

    return cluster_project_affiliation_matrix
    """

    # alternative 2
    # for each cluster, let each project affiliation be drawn from an exponential function with its own cluster-specific parameter
    cluster_project_affiliation_lower_bound = 0.1
    cluster_project_affiliation_upper_bound = 10
    # cluster_project_exponential_parameters = np.random.uniform(low=cluster_project_affiliation_lower_bound, 
    #                                                            high=cluster_project_affiliation_upper_bound, 
    #                                                            size=num_clusters)
    agent_agent_cluster_parameter_affiliation_parameter = np.mean(cluster_project_affiliation_lower_bound + cluster_project_affiliation_upper_bound)
    cluster_project_exponential_parameters = np.random.exponential(scale=agent_agent_cluster_parameter_affiliation_parameter, 
                                                                       size=num_clusters)
    
    # create the cluster-project affiliation matrix
    cluster_project_affiliation_matrix = np.zeros(shape=[num_clusters, num_projects])
    for num_cluster in range(num_clusters):
        current_cluster_parameter = cluster_project_exponential_parameters[num_cluster]
        cluster_project_affiliation_matrix[num_cluster, :] = np.random.exponential(scale=current_cluster_parameter, size=[num_projects])

    # set each value in the cluster-project affiliation matrix to 0 with probability e^-(value/temperature_scaling_factor), 
    # so that the higher the value, the lower the probability of being set to zero
    temperature_scaling_factor = np.mean(cluster_project_affiliation_lower_bound + cluster_project_affiliation_upper_bound)
    exponentiated_cluster_project_affiliation_matrix = np.exp(-cluster_project_affiliation_matrix / temperature_scaling_factor)
    mask_cluster_project_affiliation_matrix = np.random.binomial(n=np.full_like(exponentiated_cluster_project_affiliation_matrix, fill_value=1, dtype=int), 
                                                                p=exponentiated_cluster_project_affiliation_matrix,
                                                                size=exponentiated_cluster_project_affiliation_matrix.shape)

    # apply the 0-1 mask to the affiliation matrix
    cluster_project_affiliation_matrix *= mask_cluster_project_affiliation_matrix

    return cluster_project_affiliation_matrix

## Modify the Agents' Private Beliefs

Now we need a function to modify the agents' private beliefs to account for their affiliations.

In [12]:
# method 1: take into account the agents' common clusters
def modify_agent_private_beliefs(agent_private_beliefs, agent_agent_cluster_affiliation_matrix):
    """
    :param agent_private_beliefs: an array of the agents' private beliefs
    :param agent_agent_cluster_affiliation_matrix: a matrix of size num_agents x num_agents x num_clusters, where entry (i, j, k) corresponds to 
        agent i's and agent's j's affiliation on cluster k
    :return: an array of the agents' private beliefs modified to account for agent-agent-cluster affiliations 
    """

    # reduce cluster affiliations by summing up components for each agent-agent affiliation vector
    agent_agent_affiliation_matrix = agent_agent_cluster_affiliation_matrix.sum(axis=2)
    
    # # make the resulting matrix row-stochastic
    # agent_agent_affiliation_matrix_normalized = agent_agent_affiliation_matrix / agent_agent_affiliation_matrix.sum(axis=1)[:, None]
    
    # modify the agents' private beliefs to account for their affiliations with other agents 
    # through their common clusters
    agent_private_beliefs_modified = agent_agent_affiliation_matrix @ agent_private_beliefs

    return agent_private_beliefs_modified


In [13]:
# method 2: take into account the agents' common clusters as well as the affiliations between projects and clusters
def modify_agent_private_beliefs_with_project_affiliations(agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix):
    """
    :param agent_private_beliefs: an array of the agents' private beliefs
    :param agent_agent_cluster_affiliation_matrix: a matrix of size num_agents x num_agents x num_clusters, where entry (i, j, k) corresponds to 
        agent i's and agent's j's affiliation on cluster k
    :param cluster_project_affiliation_matrix: a matrix of size num_clusters x num_projects, 
        where entry (i, j) corresponds to clusters i's and project j's affiliation
    :return: an array of the agents' private beliefs modified to account for agent-agent-cluster affiliations 
    """

    # convert the agent-agent-cluster affiliations to agent-agent-project affiliations
    agent_agent_project_affiliation_matrix = agent_agent_cluster_affiliation_matrix @ cluster_project_affiliation_matrix
    # as before, reduce project affiliations by summing up components for each agent-agent affiliation vector
    agent_agent_affiliation_matrix = agent_agent_project_affiliation_matrix.sum(axis=2)
    # modify the agents' private beliefs to account for their affiliations with other agents 
    # through their common clusters
    agent_private_beliefs_modified = agent_agent_affiliation_matrix @ agent_private_beliefs

    return agent_private_beliefs_modified


In [14]:
def create_project_to_agents_to_contributions_dict_dict(agent_private_beliefs):
    """
    :param agent_private_beliefs: an array of the agents' private beliefs
    :return: a dictionary mapping a project to a dictionary mapping agents' contributions to that project
    """

    # dictionary mapping a project to the agents_to_contributions_dict corresponding to that project
    project_to_agents_to_contributions_dict_dict = {}
    # populate the dictionaries
    for project in range(num_projects):
        # initialize a dictionary mapping each agent to their contribution
        agents_to_contributions_dict = {}
        for agent_index, agent in enumerate(agents):
            agents_to_contributions_dict[agent] = agent_private_beliefs[agent_index, project]

        project_to_agents_to_contributions_dict_dict[project] = agents_to_contributions_dict
    
    return project_to_agents_to_contributions_dict_dict

In [15]:
def get_cluster_match_with_multiple_affiliations(clusters_to_agents_dict, project_to_agents_to_contributions_dict_dict):
    """
    :param clusters_to_agents_dict: a dictionary mapping clusters to the agents in those clusters
    :param project_to_agents_to_contributions_dict_dict: a dictionary mapping a project to a dictionary mapping agents' contributions to that project
    :return: an array of the cluster matches
    """
 
    cluster_matches = np.zeros(shape=num_projects)
    for project in range(num_projects):
        agents_to_contributions_dict = project_to_agents_to_contributions_dict_dict[project]
        cluster_match_with_multiple_affiliations_output = cluster_match_with_multiple_affiliations(agents_to_contributions_dict, clusters_to_agents_dict)
        # print(f"The cluster match with multiple affiliations for project {project} is {cluster_match_with_multiple_affiliations_output}")
        cluster_matches[project] = cluster_match_with_multiple_affiliations_output
        
    return cluster_matches

## Simple Simulation

Suppose we set one agent's public voice to completely copy that of another agent's. What happens?

In [16]:
def initialize_variables():
    """
    :return: initialized variables: 
             clusters_to_agents_dict, agent_budgets, agent_private_beliefs, 
             agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix
    """
    
    # create the clusters to agents dictionary
    clusters_to_agents_dict = create_clusters_to_agents_dict()
    # create the agents' budgets
    agent_budgets = create_agent_budgets()
    # create the agent's private beliefs
    agent_private_beliefs = create_agent_private_beliefs()
    # normalize the agents' private beliefs so that their sums match their budgets
    agent_private_beliefs = normalize_agent_beliefs_to_equal_budget(agent_private_beliefs, agent_budgets)
    # confirm that the above normalization worked properly
    assert(np.allclose(agent_private_beliefs.sum(axis=1), agent_budgets))
    # create the agent-agent-cluster affiliation matrix
    agent_agent_cluster_affiliation_matrix = create_agent_agent_cluster_affiliation_matrix()
    # create the cluster-project affiliation matrix
    cluster_project_affiliation_matrix = create_cluster_project_affiliation_matrix()

    return clusters_to_agents_dict, agent_budgets, agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix

In [17]:
def account_for_affiliations(agent_budgets, agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix):
    """
    :param agent_budgets: an array of the agents' budgets
    :param agent_private_beliefs: an array of the agents' private beliefs
    :param agent_agent_cluster_affiliation_matrix: a matrix of size num_agents x num_agents x num_clusters, where entry (i, j, k) corresponds to 
        agent i's and agent's j's affiliation on cluster k
    :param cluster_project_affiliation_matrix: a matrix of size num_clusters x num_projects, 
        where entry (i, j) corresponds to clusters i's and project j's affiliation
    :return: agents' private beliefs after modification and normalization
    """
    
    # modify the agents' private beliefs to account for the agent-agent-cluster affiliation matrix
    # agent_private_beliefs_modified = modify_agent_private_beliefs(agent_private_beliefs, agent_agent_cluster_affiliation_matrix)
    # modify the agents' private beliefs to account for the agent-agent-cluster affiliation matrix and the cluster-project  affiliation matrix    
    agent_private_beliefs_modified_with_project_affiliations = modify_agent_private_beliefs_with_project_affiliations(agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix)
    # normalize the agents' private beliefs so that their sums match their budgets
    agent_private_beliefs_modified_with_project_affiliations = normalize_agent_beliefs_to_equal_budget(agent_private_beliefs_modified_with_project_affiliations, agent_budgets)
    # confirm that the above normalization worked properly
    assert(np.allclose(agent_private_beliefs_modified_with_project_affiliations.sum(axis=1), agent_budgets))

    return agent_private_beliefs_modified_with_project_affiliations

In [18]:
def run_simulation():
    """
    initializes values for basic variables and computes the difference in cluster matches after selecting one agent to
    copy another agent's public voice
    """
    
    # initialize variables
    clusters_to_agents_dict, agent_budgets, agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix = initialize_variables()
    # get the agents' modified beliefs (public voices)
    agent_private_beliefs_modified_with_project_affiliations = account_for_affiliations(agent_budgets, agent_private_beliefs, agent_agent_cluster_affiliation_matrix, cluster_project_affiliation_matrix)
    # create the dictionary mapping projects to dictionaries mapping each agent's contribution to that project
    project_to_agents_to_contributions_dict_dict = create_project_to_agents_to_contributions_dict_dict(agent_private_beliefs_modified_with_project_affiliations)
    # get the cluster matches 
    cluster_match_with_multiple_affiliations_output = get_cluster_match_with_multiple_affiliations(clusters_to_agents_dict, project_to_agents_to_contributions_dict_dict)
    
    """
    suppose one agent modifies their public voice to completely align with another agent's public voice
    """
    
    # agent to modify
    agent_to_modify = np.random.randint(low=0, high=num_agents)
    # agent to copy
    agent_to_copy = agent_to_modify
    while agent_to_copy == agent_to_modify:
        agent_to_copy = np.random.randint(low=0, high=num_agents)
    # modify the agent's private belief
    agent_private_beliefs_modified_with_project_affiliations_2 = agent_private_beliefs_modified_with_project_affiliations.copy()
    agent_private_beliefs_modified_with_project_affiliations_2[agent_to_modify] = agent_private_beliefs_modified_with_project_affiliations[agent_to_copy].copy()
    
    # re-normalize the agents' private beliefs (should only impact the agent whose beliefs were modified)
    agent_private_beliefs_modified_with_project_affiliations_2 = normalize_agent_beliefs_to_equal_budget(agent_private_beliefs_modified_with_project_affiliations_2, agent_budgets)
    # confirm that the above normalization worked properly
    assert(np.allclose(agent_private_beliefs_modified_with_project_affiliations_2.sum(axis=1), agent_budgets))
    # create the dictionary mapping projects to dictionaries mapping each agent's contribution to that project
    project_to_agents_to_contributions_dict_dict_2 = create_project_to_agents_to_contributions_dict_dict(agent_private_beliefs_modified_with_project_affiliations_2)
    # get the second set of cluster matches 
    cluster_match_with_multiple_affiliations_output_2 = get_cluster_match_with_multiple_affiliations(clusters_to_agents_dict, project_to_agents_to_contributions_dict_dict_2)
    
    return cluster_match_with_multiple_affiliations_output, cluster_match_with_multiple_affiliations_output_2

In [19]:
cluster_match_with_multiple_affiliations_output_1, cluster_match_with_multiple_affiliations_output_2 = run_simulation()

In [20]:
# print the differences 
# keep in mind that these differences can vary wildly depending on how close the 
# two agents were prior to one copying the other
for num_project in range(num_projects):
    print(f"difference in cluster match for project {num_project}: {cluster_match_with_multiple_affiliations_output_1[num_project] - cluster_match_with_multiple_affiliations_output_2[num_project]}")
print()
print(f"difference in sum of cluster matches = {sum(cluster_match_with_multiple_affiliations_output_1 - cluster_match_with_multiple_affiliations_output_2)}")

difference in cluster match for project 0: -22.163867362403835
difference in cluster match for project 1: -22.417958503872796
difference in cluster match for project 2: -30.85489915985454
difference in cluster match for project 3: 2.968354807333526
difference in cluster match for project 4: 73.52457064504415
difference in cluster match for project 5: 37.132067858646224
difference in cluster match for project 6: -27.415003571819398
difference in cluster match for project 7: 12.30129669885173
difference in cluster match for project 8: -21.466667348664487
difference in cluster match for project 9: -3.494991805713653

difference in sum of cluster matches = -1.8870977424530793
